In [1]:
# Librerias generales
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import datetime
# Librerias para listar archivos
from os import listdir
from os.path import isfile, join

In [51]:
# Importo datasets

active_users = pd.read_csv('../data/ACTIVE_USER.csv') # Datos de visitas a la app
demograficos = pd.read_csv('../data/DEMOGRAFICOS.csv')
dinero_cuenta = pd.read_csv('../data/DINERO_CUENTA.csv') # Datos de la cantidad de dinero en la cuenta y si invierta
evaluate = pd.read_csv('../data/EVALUATE.csv')
marketplace_data = pd.read_csv('../data/MARKETPLACE_DATA.csv') # Movimientos en mercadolibre
payments = pd.read_csv('../data/PAYMENTS.csv') # Datos transaccionales en MP

In [52]:
# Active users
active_users.head(3)

,CUS_CUST_ID_BUY,MAU_MP_3,MAU_ML_3,MAU_MP_2,MAU_ML_2,MAU_MP_1,MAU_ML_1,last_login_mp_date_1,last_login_ml_date_1
0,424070401,NaN,NaN,1.0,0.0,NaN,NaN,NaN,NaN
1,271894453,1.0,14.0,11.0,21.0,2.0,12.0,2019-05-17,2019-05-24
2,327402711,NaN,NaN,0.0,10.0,4.0,26.0,2019-05-28,2019-05-31


In [53]:
# Demograficos
demograficos.head(3)

,CITY,CUS_CUST_ID_BUY,GENDER,RANGO_EDAD,TARJETAS,ESTADO
0,caba,47629682,male,03.Entre 26 y 30 años,Credit Card,capital federal
1,belen de escobar,184288617,male,03.Entre 26 y 30 años,Account Money,buenos aires
2,san fernando del valle de catamarca,134359455,female,03.Entre 26 y 30 años,Debit Card,catamarca


In [54]:
# Dinero en cuenta
dinero_cuenta.head(3)

,CUS_CUST_ID_BUY,PLATA_CUENTA_1,PLATA_CUENTA_2,INVERSION
0,54018.0,250.99,0.00,pending
1,23513.0,20.05,2220.05,warmup
2,21230.0,0.00,0.00,warmup


In [55]:
# Evaluate
evaluate.head(3)

,CUS_CUST_ID_BUY,churn
0,20663512,1.0
1,444286946,0.0
2,348084528,1.0


In [56]:
# Marketplace 
marketplace_data.head(3)

,CUS_CUST_ID_BUY,SPENT_ML,RECENCY_ML,FREQUENCY_ML
0,53621056,71.51,2019-03-30,1
1,308208758,269.20,2019-05-30,11
2,30591970,149.22,2019-03-24,4


In [57]:
# Payments
payments.head(3)

,FECHA,CUS_CUST_ID_SEL,CUS_CUST_ID_BUY,SPENT,TPV_SEGMENT_DETAIL,DESCUENTO
0,2019-05-25,314941456,20663512,11.3566,Instore,2.2312
1,2019-06-20,251693291,444286946,4.6190,Transport,0.9238
2,2019-01-10,251693291,20663512,5.3519,Transport,1.0704


In [58]:
# Revision de payments dataframe

payments.info() # La información esta ok con sus clases excepto por la fecha, se convierte a datetime

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555516 entries, 0 to 555515
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   FECHA               555516 non-null  object 
 1   CUS_CUST_ID_SEL     555516 non-null  int64  
 2   CUS_CUST_ID_BUY     555516 non-null  int64  
 3   SPENT               555516 non-null  float64
 4   TPV_SEGMENT_DETAIL  555516 non-null  object 
 5   DESCUENTO           555516 non-null  float64
dtypes: float64(2), int64(2), object(2)
memory usage: 25.4+ MB


In [59]:
# Convierto la columna fecha en datetime
payments['FECHA'] = [datetime.datetime.strptime(x, '%Y-%m-%d') for x in payments['FECHA']]

In [60]:
# La idea es unificar la data por usuario de forma que cada linea tenga toda la información
# Por eso primero se dummiza el dataframe payments para que los distintos segmentos estén por columna

payments = pd.get_dummies(payments, columns = ['TPV_SEGMENT_DETAIL'], prefix = ['segment'])

In [70]:
payments.groupby([pd.Grouper(key = 'FECHA', freq = '3M'), 'CUS_CUST_ID_BUY'], as_index = True).agg(spent = ('SPENT', 'sum'), spent_count = ('SPENT', 'count'), spent_mean = ('SPENT', 'mean'), discount = ('DESCUENTO','sum')) 

spent  spent_count  spent_mean  discount
FECHA      CUS_CUST_ID_BUY                                             
2019-01-31 14525             14.2774            2    7.138700    2.8555
           22083             69.0491            3   23.016367    7.7778
           34173            136.9079            2   68.453950    0.0000
           36482             23.9667            5    4.793340    0.0000
           51395             36.4893           11    3.317209    1.0276
...                              ...          ...         ...       ...
2019-07-31 448290283          4.7100            1    4.710000    0.0000
           448313152          4.7100            1    4.710000    0.0000
           448326912          7.7715            1    7.771500    1.5543
           448330186          9.7025            1    9.702500    1.9405
           448374921         17.7095            1   17.709500    3.5325

[77028 rows x 4 columns]

In [91]:
# Funcion de agregacion por ventanas de tiempo
payments_group_date = payments.groupby([pd.Grouper(key = 'FECHA', freq = '3M'), 'CUS_CUST_ID_BUY'], as_index = True).agg(spent = ('SPENT', 'sum'), 
                                                                                                                         spent_count = ('SPENT', 'count'), 
                                                                                                                         spent_mean = ('SPENT', 'mean'), 
                                                                                                                         discount = ('DESCUENTO','sum'),
                                                                                                                         discount_mean = ('DESCUENTO','mean'))  
payments_group_date = payments_group_date.reset_index() 
payments_group_date['MES'] = [str(x.month) for x in payments_group_date['FECHA']]
k = len(payments_group_date['MES'].unique())
payments_group_date = pd.get_dummies(payments_group_date, columns = ['MES'])
max_index = payments_group_date.shape[1] - 1
low_index = max_index  + 1 - k

payments_group_date

for col1 in payments_group_date.columns[2:low_index]:
    for col2 in payments_group_date.columns[low_index:max_index + 1]:
        col_name = col1 + '_' + col2
        payments_group_date[col_name] = payments_group_date[col2].multiply(payments_group_date[col1], axis = 'index')

In [92]:
payments_group_date

,FECHA,CUS_CUST_ID_BUY,spent,spent_count,spent_mean,discount,discount_mean,MES_1,MES_4,MES_7,...,spent_count_MES_7,spent_mean_MES_1,spent_mean_MES_4,spent_mean_MES_7,discount_MES_1,discount_MES_4,discount_MES_7,discount_mean_MES_1,discount_mean_MES_4,discount_mean_MES_7
0,2019-01-31,14525,14.2774,2,7.138700,2.8555,1.427750,1,0,0,...,0,7.138700,0.0,0.0000,2.8555,0.0,0.0000,1.427750,0.0,0.0000
1,2019-01-31,22083,69.0491,3,23.016367,7.7778,2.592600,1,0,0,...,0,23.016367,0.0,0.0000,7.7778,0.0,0.0000,2.592600,0.0,0.0000
2,2019-01-31,34173,136.9079,2,68.453950,0.0000,0.000000,1,0,0,...,0,68.453950,0.0,0.0000,0.0000,0.0,0.0000,0.000000,0.0,0.0000
3,2019-01-31,36482,23.9667,5,4.793340,0.0000,0.000000,1,0,0,...,0,4.793340,0.0,0.0000,0.0000,0.0,0.0000,0.000000,0.0,0.0000
4,2019-01-31,51395,36.4893,11,3.317209,1.0276,0.093418,1,0,0,...,0,3.317209,0.0,0.0000,1.0276,0.0,0.0000,0.093418,0.0,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77023,2019-07-31,448290283,4.7100,1,4.710000,0.0000,0.000000,0,0,1,...,1,0.000000,0.0,4.7100,0.0000,0.0,0.0000,0.000000,0.0,0.0000
77024,2019-07-31,448313152,4.7100,1,4.710000,0.0000,0.000000,0,0,1,...,1,0.000000,0.0,4.7100,0.0000,0.0,0.0000,0.000000,0.0,0.0000
77025,2019-07-31,448326912,7.7715,1,7.771500,1.5543,1.554300,0,0,1,...,1,0.000000,0.0,7.7715,0.0000,0.0,1.5543,0.000000,0.0,1.5543
77026,2019-07-31,448330186,9.7025,1,9.702500,1.9405,1.940500,0,0,1,...,1,0.000000,0.0,9.7025,0.0000,0.0,1.9405,0.000000,0.0,1.9405


In [62]:
# Funcion de agregacion por ventanas de tiempo. La idea es obtener distintos valores de agregacion para distintas ventanas temporales
# por ejemplo gasto en los ultimos tres meses, seis meses, etc.

# Funcion de agregacion por ventanas de tiempo
payments_group_date = payments.groupby([pd.Grouper(key = 'FECHA', freq = '3M'), 'CUS_CUST_ID_BUY'], as_index = True).agg(spent = ('SPENT', 'sum'), 
                                                                                                                         spent_count = ('SPENT', 'count'), 
                                                                                                                         spent_mean = ('SPENT', 'mean'), 
                                                                                                                         discount = ('DESCUENTO','sum'),
                                                                                                                         discount_mean = ('DESCUENTO','mean'))  
payments_group_date = payments_group_date.reset_index()

# Creo un feature indicando el mes para poder crear columnas dummies con esta variable y asi obtener lo gastado y el descuento en esos meses
payments_group_date['MES'] = [str(x.month) for x in payments_group_date['FECHA']]
k = len(payments_group_date['MES'].unique())

# Spent 
payments_group_date_spent = pd.get_dummies(payments_group_date, columns = ['MES'])
payments_group_date_spent.iloc[:,-k:] = payments_group_date_spent.iloc[:,-k:].multiply(payments_group_date_spent['spent'], axis = 'index')

# Spent count
payments_group_date_spent_count = pd.get_dummies(payments_group_date, columns = ['MES'])
payments_group_date_spent_count.iloc[:,-k:] = payments_group_date_spent_count.iloc[:,-k:].multiply(payments_group_date_spent_count['spent_count'], axis = 'index')

# Spent mean
payments_group_date_spent_mean = pd.get_dummies(payments_group_date, columns = ['MES'])
payments_group_date_spent_mean.iloc[:,-k:] = payments_group_date_spent_mean.iloc[:,-k:].multiply(payments_group_date_spent_mean['spent_mean'], axis = 'index')

# Discount 
payments_group_date_discount = pd.get_dummies(payments_group_date, columns = ['MES'])
payments_group_date_discount.iloc[:,-k:] = payments_group_date_discount.iloc[:,-k:].multiply(payments_group_date_discount['discount'], axis = 'index')

# Discount mean
payments_group_date_discount_mean = pd.get_dummies(payments_group_date, columns = ['MES'])
payments_group_date_discount_mean.iloc[:,-k:] = payments_group_date_discount_mean.iloc[:,-k:].multiply(payments_group_date_discount['discount_mean'], axis = 'index')

# Paso final es agrupar todo por usuario para obtener registros unicos
# Spent
payments_group_date_spent = payments_group_date_spent.groupby('CUS_CUST_ID_BUY', as_index = False).sum()
payments_group_date_spent.drop(columns = ['spent','spent_count','spent_mean','discount'], axis = 1, inplace = True)

# Spent Count
payments_group_date_spent_count = payments_group_date_spent_count.groupby('CUS_CUST_ID_BUY', as_index = False).sum()
payments_group_date_spent_count.drop(columns = ['spent','spent_count','spent_mean','discount'], axis = 1, inplace = True)

# Spent Mean
payments_group_date_spent_mean = payments_group_date_spent_mean.groupby('CUS_CUST_ID_BUY', as_index = False).sum()
payments_group_date_spent_mean.drop(columns = ['spent','spent_count','spent_mean','discount'], axis = 1, inplace = True)

# Discount
payments_group_date_discount = payments_group_date_discount.groupby('CUS_CUST_ID_BUY', as_index = False).sum()
payments_group_date_discount.drop(columns = ['spent','spent_count','spent_mean','discount'], axis = 1, inplace = True)

# Discount mean
payments_group_date_discount_mean = payments_group_date_discount_mean.groupby('CUS_CUST_ID_BUY', as_index = False).sum()
payments_group_date_discount_mean.drop(columns = ['spent','spent_count','spent_mean','discount'], axis = 1, inplace = True)


In [115]:
payments_group_spent = payments.groupby('CUS_CUST_ID_BUY', as_index = False).agg({'SPENT':['sum','mean','count'], 'DESCUENTO':['sum','mean']},)
payments_group_spent.columns = ['_'.join(col).strip() for col in payments_group_spent.columns.values]
payments_group_spent.rename(columns = {'CUS_CUST_ID_BUY_':'CUS_CUST_ID_BUY'}, inplace = True)        
payments_group_spent                                                           

,CUS_CUST_ID_BUY,SPENT_sum,SPENT_mean,SPENT_count,DESCUENTO_sum,DESCUENTO_mean
0,1276,7.6365,1.909125,4,0.0000,0.000000
1,14525,429.3148,9.332930,46,18.6575,0.405598
2,21230,4.4198,4.419800,1,2.2099,2.209900
3,22083,196.6036,10.922422,18,14.9465,0.830361
4,23513,31.1145,3.111450,10,0.0000,0.000000
...,...,...,...,...,...,...
43497,448290283,4.7100,4.710000,1,0.0000,0.000000
43498,448313152,4.7100,4.710000,1,0.0000,0.000000
43499,448326912,7.7715,7.771500,1,1.5543,1.554300
43500,448330186,9.7025,9.702500,1,1.9405,1.940500


In [120]:
dates = payments_group_date['FECHA'].unique()
subset 1= payments_group_date[payments_group_date['FECHA'] == [dates0]]
[['CUS_CUST_ID_BUY','spent','discount']]

subset2 = payments_group_date[payments_group_date['FECHA'] == dates[1]][['CUS_CUST_ID_BUY','spent','discount']]
psubset3 = payments_group_date[payments_group_date['FECHA'] == dates[2]][['CUS_CUST_ID_BUY','spent','discount']]d.merge(payments_group_spent,  subset on = 'CUS_CUST_ID_BUY', how = 'left')

,CUS_CUST_ID_BUY,SPENT_sum,SPENT_mean,SPENT_count,DESCUENTO_sum,DESCUENTO_mean,spent,discount
0,1276,7.6365,1.909125,4,0.0000,0.000000,NaN,NaN
1,14525,429.3148,9.332930,46,18.6575,0.405598,14.2774,2.8555
2,21230,4.4198,4.419800,1,2.2099,2.209900,NaN,NaN
3,22083,196.6036,10.922422,18,14.9465,0.830361,69.0491,7.7778
4,23513,31.1145,3.111450,10,0.0000,0.000000,NaN,NaN
...,...,...,...,...,...,...,...,...
43497,448290283,4.7100,4.710000,1,0.0000,0.000000,NaN,NaN
43498,448313152,4.7100,4.710000,1,0.0000,0.000000,NaN,NaN
43499,448326912,7.7715,7.771500,1,1.5543,1.554300,NaN,NaN
43500,448330186,9.7025,9.702500,1,1.9405,1.940500,NaN,NaN


In [117]:
payments[payments['CUS_CUST_ID_BUY'] == 1276]

,FECHA,CUS_CUST_ID_SEL,CUS_CUST_ID_BUY,SPENT,DESCUENTO,segment_Cellphone Recharge,segment_Instore,segment_Money Transfer,segment_Other Single Players,segment_Transport,segment_Utilities
372096,2019-03-20,295153733,1276,1.2500,0.0,1,0,0,0,0,0
373227,2019-02-16,295153733,1276,2.5893,0.0,1,0,0,0,0,0
375585,2019-05-07,295153733,1276,1.1198,0.0,1,0,0,0,0,0
377042,2019-02-01,295153733,1276,2.6774,0.0,1,0,0,0,0,0
